<a href="https://colab.research.google.com/github/samiemostafavi/ros-latency-pr3d/blob/main/ros_cross_traffic_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Login to Chameleon and download openrc.sh file from [here](https://testbed.expeca.proj.kth.se/project/api_access/openrc/). Upload it here next to this notebook and continue.

In the next cell, we setup the authentication method to be able to use Openstack clients.

In [1]:
import os, re
from getpass import getpass

with open('bryan-project-openrc.sh', 'r') as f:
    script_content = f.read()
    pattern = r'export\s+(\w+)\s*=\s*("[^"]+"|[^"\n]+)'
    matches = re.findall(pattern, script_content)

    for name, value in matches:
        os.environ[name] = value.strip('"')

password = getpass('enter your expeca password:')
os.environ['OS_PASSWORD'] = password

enter your expeca password:··········


Install required packages and dependencies. Ignore the warnings.

In [2]:
!pip uninstall -q -y moviepy
!pip install -q jedi
!pip install -q git+https://github.com/KTH-EXPECA/python-chi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 32.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.3/227.3 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.6/341.6 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.0/207.0 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.0/241.0 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.9/517.9 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

Import packages

In [3]:
import json
from loguru import logger
import chi.network, chi.container
from chi.expeca import reserve, list_reservations, unreserve_byid, get_container_status, wait_until_container_removed, show_reservation_byname, get_available_publicips, get_segment_ids, get_worker_interfaces

In the next cell, we reserve the required equipment and resources to form an end to end experiment setup. We reserve EP5G network.

In [6]:
# ep5g reservation
ep5g_lease = show_reservation_byname("ep5g-lease")
segment_ids = get_segment_ids('ep5g')
if not ep5g_lease:
    ep5g_lease = reserve(
        { "type":"network", "name": "ep5g", "net_name": "ep5g-vip", "segment_id": segment_ids['rj45'], "duration": { "days":7, "hours":0 } }
    )


leaseslist = list_reservations(brief=True)
print(json.dumps(leaseslist,indent=4))

2025-02-20 10:46:15.728 | INFO     | chi.expeca:reserve:243 - reserving ep5g
2025-02-20 10:46:18.515 | INFO     | chi.expeca:wait_until_lease_status:138 - waiting 120 seconds for ep5g-lease with id 82dbd038-b461-4fab-aeab-74e0b0827036 to become "ACTIVE"
2025-02-20 10:46:23.669 | INFO     | chi.expeca:wait_until_lease_status:145 - lease ep5g-lease with id 82dbd038-b461-4fab-aeab-74e0b0827036 is PENDING.
2025-02-20 10:46:28.822 | INFO     | chi.expeca:wait_until_lease_status:145 - lease ep5g-lease with id 82dbd038-b461-4fab-aeab-74e0b0827036 is PENDING.
2025-02-20 10:46:33.971 | INFO     | chi.expeca:wait_until_lease_status:145 - lease ep5g-lease with id 82dbd038-b461-4fab-aeab-74e0b0827036 is PENDING.
2025-02-20 10:46:39.129 | INFO     | chi.expeca:wait_until_lease_status:145 - lease ep5g-lease with id 82dbd038-b461-4fab-aeab-74e0b0827036 is PENDING.
2025-02-20 10:46:44.267 | INFO     | chi.expeca:wait_until_lease_status:145 - lease ep5g-lease with id 82dbd038-b461-4fab-aeab-74e0b082703

[
    {
        "name": "ep5g-lease",
        "id": "594e761f-845a-4d5b-bca6-e2fe2e685372",
        "reservation_id": "ed2f4280-9e42-4d71-af7e-459388779970",
        "status": "ACTIVE",
        "end_date": "2025-02-27T10:46:00.000000"
    }
]


In the following section we setup the networking equipment for ep5g, edge-net, Telenor Edge, and Internet (refer to [here](https://kth-expeca.gitbook.io/testbedconfig/enroll/enroll-network-segments/ep5g) for more info).

In [7]:
# create edge-net
edgenet = chi.network.create_network("edge-net")
chi.network.create_subnet("edge-net-subnet",edgenet["id"],"10.70.70.0/24",gateway_ip=None,enable_dhcp=False)
logger.success("edge-net is created.")

# create ep5g-vip-router
router = chi.network.create_router("ep5g-vip-router", "public")
logger.success("ep5g-vip-router router is created.")
logger.info(f"{json.dumps(router,indent=4)}")

# connect ep5g-vip-net to ep5g-vip-router
ep5gnet = chi.network.get_network("ep5g-vip-net")
portadd = chi.network.add_subnet_to_router(router["id"],ep5gnet["subnets"][0])
logger.success("An interface on ep5g-vip-net is added to the router")

# connect edge-net to ep5g-vip-router
edgenet = chi.network.get_network("edge-net")
portadd = chi.network.add_subnet_to_router(router["id"],edgenet["subnets"][0])
logger.success("An interface on edge-net is added to the router")

# add ep5g route to ep5g-vip-router
routeadd = chi.network.add_route_to_router(router["id"],"172.16.0.0/16","10.30.111.10")
logger.success("the ep5g route added to the router")

2025-02-20 10:48:06.720 | SUCCESS  | __main__:<cell line: 0>:4 - edge-net is created.
2025-02-20 10:48:09.222 | SUCCESS  | __main__:<cell line: 0>:8 - ep5g-vip-router router is created.
2025-02-20 10:48:09.223 | INFO     | __main__:<cell line: 0>:9 - {
    "id": "caa606b5-0318-4b93-8d11-12f513e1dc65",
    "name": "ep5g-vip-router",
    "tenant_id": "c47f3ed65941413fbc6087028aff5ee2",
    "admin_state_up": true,
    "status": "ACTIVE",
    "external_gateway_info": {
        "network_id": "717b5f2b-069e-4868-a24d-91a4ae3ad002",
        "external_fixed_ips": [
            {
                "subnet_id": "53d03ffd-0d49-4f9a-88a3-a30d69fe4827",
                "ip_address": "130.237.11.102"
            }
        ],
        "enable_snat": true
    },
    "description": "",
    "availability_zones": [],
    "availability_zone_hints": [],
    "routes": [],
    "flavor_id": null,
    "tags": [],
    "created_at": "2025-02-20T10:48:07Z",
    "updated_at": "2025-02-20T10:48:08Z",
    "revision_num

In [8]:
# worker reservation
worker_name = "worker-06"
worker_lease = show_reservation_byname(worker_name + "-lease")
if not worker_lease:
    worker_lease = reserve(
        { "type":"device", "name":worker_name, "duration": { "days":7, "hours":0 } }
    )
worker_reservation_id = worker_lease["reservations"][0]["id"]

# worker reservation
worker_name = "worker-07"
worker_lease = show_reservation_byname(worker_name + "-lease")
if not worker_lease:
    worker_lease = reserve(
        { "type":"device", "name":worker_name, "duration": { "days":7, "hours":0 } }
    )
worker_reservation_id = worker_lease["reservations"][0]["id"]

# worker reservation
worker_name = "worker-08"
worker_lease = show_reservation_byname(worker_name + "-lease")
if not worker_lease:
    worker_lease = reserve(
        { "type":"device", "name":worker_name, "duration": { "days":7, "hours":0 } }
    )
worker_reservation_id = worker_lease["reservations"][0]["id"]

# worker reservation
worker_name = "worker-09"
worker_lease = show_reservation_byname(worker_name + "-lease")
if not worker_lease:
    worker_lease = reserve(
        { "type":"device", "name":worker_name, "duration": { "days":7, "hours":0 } }
    )
worker_reservation_id = worker_lease["reservations"][0]["id"]

2025-02-20 10:48:18.185 | INFO     | chi.expeca:reserve:243 - reserving worker-06
2025-02-20 10:48:20.622 | INFO     | chi.expeca:wait_until_lease_status:138 - waiting 120 seconds for worker-06-lease with id 3938909a-4875-4242-ac7d-26f07167a101 to become "ACTIVE"
2025-02-20 10:48:25.785 | INFO     | chi.expeca:wait_until_lease_status:145 - lease worker-06-lease with id 3938909a-4875-4242-ac7d-26f07167a101 is PENDING.
2025-02-20 10:48:30.928 | INFO     | chi.expeca:wait_until_lease_status:145 - lease worker-06-lease with id 3938909a-4875-4242-ac7d-26f07167a101 is PENDING.
2025-02-20 10:48:36.070 | INFO     | chi.expeca:wait_until_lease_status:145 - lease worker-06-lease with id 3938909a-4875-4242-ac7d-26f07167a101 is PENDING.
2025-02-20 10:48:41.220 | INFO     | chi.expeca:wait_until_lease_status:145 - lease worker-06-lease with id 3938909a-4875-4242-ac7d-26f07167a101 is PENDING.
2025-02-20 10:48:46.359 | INFO     | chi.expeca:wait_until_lease_status:145 - lease worker-06-lease with id 

In [9]:
# advantech router reservation
adv_name = "adv-02"
segment_ids = get_segment_ids(adv_name)
adv_lease = show_reservation_byname(adv_name + "-lease")
if not adv_lease:
    adv_lease = reserve(
        { "type":"network", "name": adv_name, "net_name": adv_name, "segment_id": segment_ids['rj45'], "duration": { "days":7, "hours":0 } }
    )

# advantech router reservation
adv_name = "adv-03"
segment_ids = get_segment_ids(adv_name)
adv_lease = show_reservation_byname(adv_name + "-lease")
if not adv_lease:
    adv_lease = reserve(
        { "type":"network", "name": adv_name, "net_name": adv_name, "segment_id": segment_ids['rj45'], "duration": { "days":7, "hours":0 } }
    )

# advantech router reservation
adv_name = "adv-04"
segment_ids = get_segment_ids(adv_name)
adv_lease = show_reservation_byname(adv_name + "-lease")
if not adv_lease:
    adv_lease = reserve(
        { "type":"network", "name": adv_name, "net_name": adv_name, "segment_id": segment_ids['rj45'], "duration": { "days":7, "hours":0 } }
    )

# advantech router reservation
adv_name = "adv-05"
segment_ids = get_segment_ids(adv_name)
adv_lease = show_reservation_byname(adv_name + "-lease")
if not adv_lease:
    adv_lease = reserve(
        { "type":"network", "name": adv_name, "net_name": adv_name, "segment_id": segment_ids['rj45'], "duration": { "days":7, "hours":0 } }
    )

# advantech router reservation
adv_name = "adv-06"
segment_ids = get_segment_ids(adv_name)
adv_lease = show_reservation_byname(adv_name + "-lease")
if not adv_lease:
    adv_lease = reserve(
        { "type":"network", "name": adv_name, "net_name": adv_name, "segment_id": segment_ids['rj45'], "duration": { "days":7, "hours":0 } }
    )

# advantech router reservation
adv_name = "adv-07"
segment_ids = get_segment_ids(adv_name)
adv_lease = show_reservation_byname(adv_name + "-lease")
if not adv_lease:
    adv_lease = reserve(
        { "type":"network", "name": adv_name, "net_name": adv_name, "segment_id": segment_ids['rj45'], "duration": { "days":7, "hours":0 } }
    )

2025-02-20 10:52:19.362 | INFO     | chi.expeca:reserve:243 - reserving adv-02
2025-02-20 10:52:21.915 | INFO     | chi.expeca:wait_until_lease_status:138 - waiting 120 seconds for adv-02-lease with id 3d82c4d0-3897-4734-ba7a-e54be33a960e to become "ACTIVE"
2025-02-20 10:52:27.055 | INFO     | chi.expeca:wait_until_lease_status:145 - lease adv-02-lease with id 3d82c4d0-3897-4734-ba7a-e54be33a960e is PENDING.
2025-02-20 10:52:32.215 | INFO     | chi.expeca:wait_until_lease_status:145 - lease adv-02-lease with id 3d82c4d0-3897-4734-ba7a-e54be33a960e is PENDING.
2025-02-20 10:52:37.373 | INFO     | chi.expeca:wait_until_lease_status:145 - lease adv-02-lease with id 3d82c4d0-3897-4734-ba7a-e54be33a960e is PENDING.
2025-02-20 10:52:42.536 | INFO     | chi.expeca:wait_until_lease_status:145 - lease adv-02-lease with id 3d82c4d0-3897-4734-ba7a-e54be33a960e is PENDING.
2025-02-20 10:52:47.690 | INFO     | chi.expeca:wait_until_lease_status:145 - lease adv-02-lease with id 3d82c4d0-3897-4734-ba

In [10]:
# worker reservation
worker_name = "worker-07"
worker_lease = show_reservation_byname(worker_name + "-lease")
worker_reservation_id = worker_lease["reservations"][0]["id"]
edgenet = chi.network.get_network("edge-net")
container_name = "ct-server-node"
chi.container.create_container(
    name = container_name,
    image = "samiemostafavi/perf-meas",
    reservation_id = worker_reservation_id,
    environment = {"SERVER_DIR":"/tmp/"},
    mounts = [],
    nets = [
        { "network" : edgenet['id'] },
    ],
    labels = {
        "networks.1.interface":"ens1",
        "networks.1.ip":"10.70.70.210/24",
        "networks.1.routes":"172.16.0.0/16-10.70.70.1",
    },
)
chi.container.wait_for_active(container_name)
logger.success(f"created {container_name} container.")

2025-02-20 10:59:21.886 | SUCCESS  | __main__:<cell line: 0>:23 - created ct-server-node container.


In [11]:
ports = [
    53301,
    53302,
    53303,
    53304,
    53305
]
container_name = "ct-server-node"
for port in ports:
    command = f"iperf3 -s -p {port}"
    result = chi.container.execute(
        container_ref=container_name,
        command="curl -s -X POST -H \"Content-Type: application/json\" -d '{\"cmd\": \"" + command + "\"}' http://localhost:50505/",
    )
    logger.info(f"{result}")

2025-02-20 10:59:27.273 | INFO     | __main__:<cell line: 0>:15 - {'output': 'Command received and started in the background.\n', 'exit_code': 0, 'exec_id': None, 'proxy_url': None}
2025-02-20 10:59:27.575 | INFO     | __main__:<cell line: 0>:15 - {'output': 'Command received and started in the background.\n', 'exit_code': 0, 'exec_id': None, 'proxy_url': None}
2025-02-20 10:59:27.902 | INFO     | __main__:<cell line: 0>:15 - {'output': 'Command received and started in the background.\n', 'exit_code': 0, 'exec_id': None, 'proxy_url': None}
2025-02-20 10:59:28.236 | INFO     | __main__:<cell line: 0>:15 - {'output': 'Command received and started in the background.\n', 'exit_code': 0, 'exec_id': None, 'proxy_url': None}
2025-02-20 10:59:28.528 | INFO     | __main__:<cell line: 0>:15 - {'output': 'Command received and started in the background.\n', 'exit_code': 0, 'exec_id': None, 'proxy_url': None}


In [12]:
advnet = chi.network.get_network("adv-02-net")
container_name = "ct-client-node-01"
chi.container.create_container(
    name = container_name,
    image = "samiemostafavi/perf-meas",
    reservation_id = worker_reservation_id,
    environment = {"SERVER_DIR":"/tmp/"},
    nets = [
        { "network" : advnet['id'] },
    ],
    labels = {
        "networks.1.interface":"eno12409",
        "networks.1.ip":"10.42.3.2/24",
        "networks.1.routes":"10.70.70.0/24-10.42.3.1",
    },
)
chi.container.wait_for_active(container_name)
logger.success(f"created {container_name} container.")

2025-02-20 11:00:58.509 | SUCCESS  | __main__:<cell line: 0>:18 - created ct-client-node-01 container.


In [13]:
advnet = chi.network.get_network("adv-03-net")
container_name = "ct-client-node-02"
chi.container.create_container(
    name = container_name,
    image = "samiemostafavi/perf-meas",
    reservation_id = worker_reservation_id,
    environment = {"SERVER_DIR":"/tmp/"},
    nets = [
        { "network" : advnet['id'] },
    ],
    labels = {
        "networks.1.interface":"eno12419",
        "networks.1.ip":"10.42.3.2/24",
        "networks.1.routes":"10.70.70.0/24-10.42.3.1",
    },
)
chi.container.wait_for_active(container_name)
logger.success(f"created {container_name} container.")

2025-02-20 11:01:33.105 | SUCCESS  | __main__:<cell line: 0>:18 - created ct-client-node-02 container.


In [14]:
advnet = chi.network.get_network("adv-04-net")
container_name = "ct-client-node-03"
chi.container.create_container(
    name = container_name,
    image = "samiemostafavi/perf-meas",
    reservation_id = worker_reservation_id,
    environment = {"SERVER_DIR":"/tmp/"},
    nets = [
        { "network" : advnet['id'] },
    ],
    labels = {
        "networks.1.interface":"eno12429",
        "networks.1.ip":"10.42.3.2/24",
        "networks.1.routes":"10.70.70.0/24-10.42.3.1",
    },
)
chi.container.wait_for_active(container_name)
logger.success(f"created {container_name} container.")

2025-02-20 11:02:06.356 | SUCCESS  | __main__:<cell line: 0>:18 - created ct-client-node-03 container.


In [15]:
worker_name = "worker-08"
worker_lease = show_reservation_byname(worker_name + "-lease")
worker_reservation_id = worker_lease["reservations"][0]["id"]

advnet = chi.network.get_network("adv-05-net")
container_name = "ct-client-node-04"
chi.container.create_container(
    name = container_name,
    image = "samiemostafavi/perf-meas",
    reservation_id = worker_reservation_id,
    environment = {"SERVER_DIR":"/tmp/"},
    nets = [
        { "network" : advnet['id'] },
    ],
    labels = {
        "networks.1.interface":"ens1",
        "networks.1.ip":"10.42.3.2/24",
        "networks.1.routes":"10.70.70.0/24-10.42.3.1",
    },
)
chi.container.wait_for_active(container_name)
logger.success(f"created {container_name} container.")

2025-02-20 11:02:34.719 | SUCCESS  | __main__:<cell line: 0>:22 - created ct-client-node-04 container.


In [16]:
advnet = chi.network.get_network("adv-06-net")
container_name = "ct-client-node-05"
chi.container.create_container(
    name = container_name,
    image = "samiemostafavi/perf-meas",
    reservation_id = worker_reservation_id,
    environment = {"SERVER_DIR":"/tmp/"},
    nets = [
        { "network" : advnet['id'] },
    ],
    labels = {
        "networks.1.interface":"eno12409",
        "networks.1.ip":"10.42.3.2/24",
        "networks.1.routes":"10.70.70.0/24-10.42.3.1",
    },
)
chi.container.wait_for_active(container_name)
logger.success(f"created {container_name} container.")

2025-02-20 11:02:56.424 | SUCCESS  | __main__:<cell line: 0>:18 - created ct-client-node-05 container.


In [17]:
worker_name = "worker-06"
worker_lease = show_reservation_byname(worker_name + "-lease")
worker_reservation_id = worker_lease["reservations"][0]["id"]

publicnet = chi.network.get_network("serverpublic")
edgenet = chi.network.get_network("edge-net")
container_name = "ros-latency-server-node"
chi.container.create_container(
    name = container_name,
    image = "bryandony/ros:melodic-voxblox-edge-expeca",
    reservation_id = worker_reservation_id,
    environment = {
        "DNS_IP":"8.8.8.8",
        "GATEWAY_IP":"130.237.11.97",
        "PASS":"expeca"
    },
    mounts = [],
    nets = [
        { "network" : publicnet['id'] },
        { "network" : edgenet['id'] },
    ],
    labels = {
        "networks.1.interface":"ens1f1",
        "networks.1.ip":"130.237.11.122/27",
        "networks.1.gateway":"130.237.11.97",
        "networks.2.interface":"eno12399np0",
        "networks.2.ip":"10.70.70.211/24",
        "networks.2.routes":"172.16.0.0/16-10.70.70.1",
        "capabilities.privileged":"true",
    },
)
chi.container.wait_for_active(container_name)
logger.success(f"created {container_name} container.")

2025-02-20 11:04:09.370 | SUCCESS  | __main__:<cell line: 0>:33 - created ros-latency-server-node container.


In [18]:
worker_name = "worker-09"
worker_lease = show_reservation_byname(worker_name + "-lease")
worker_reservation_id = worker_lease["reservations"][0]["id"]
publicnet = chi.network.get_network("serverpublic")
advnet = chi.network.get_network("adv-07-net")
name = "ros-latency-client-node"
chi.container.create_container(
    name = name,
    image = "bryandony/ros:melodic-voxblox-edge-expeca",
    reservation_id = worker_reservation_id,
    environment = {"SERVER_DIR":"/tmp/"},
    nets = [
        { "network" : publicnet['id'] },
        { "network" : advnet['id'] },
    ],
    labels = {
        "networks.1.interface":"ens1",
        "networks.1.ip":"130.237.11.123/27",
        "networks.1.gateway":"130.237.11.97",
        "networks.2.interface":"eno12409",
        "networks.2.ip":"10.42.3.2/24",
        "networks.2.routes":"10.70.70.0/24-10.42.3.1",
        "capabilities.privileged":"true",
    },
)
chi.container.wait_for_active(name)
logger.success(f"created {name} container.")

2025-02-20 11:04:44.366 | SUCCESS  | __main__:<cell line: 0>:27 - created ros-latency-client-node container.


In [ ]:
ports = [
    53301,
    53302,
    53303,
    53304,
    53305
]
nodes_names = [
    "ct-client-node-01",
    "ct-client-node-02",
    "ct-client-node-03",
    "ct-client-node-04",
    "ct-client-node-05"
]

for container_name,port in zip(nodes_names,ports):
    command = f"iperf3 -c 10.70.70.210 -p {port} -b 5M -t 30"
    result = chi.container.execute(
        container_ref=container_name,
        command="curl -s -X POST -H \"Content-Type: application/json\" -d '{\"cmd\": \"" + command + "\"}' http://localhost:50505/",
    )
    logger.info(f"{result}")